# Social Media Monitoring


The following table contains data on the politicians and their associated social acounts

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
from datetime import datetime
import os
from smm_wrapper import SMM
import qgrid
smm = SMM()

politicians_df = smm.dv.politicians_df()
qgrid_widget = qgrid.show_grid(politicians_df)
qgrid_widget

In [ ]:
from collections import defaultdict 
party_list = []
for name, group in politicians_df.groupby(['affiliation']):
    party_list.append({'party':name,'labels':[],'values':[]})
#print(party_list)

In [ ]:
def search(party, party_list):
    for element in party_list:
        if element['party'] == party:
            return dict(element)

for index, row in politicians_df.iterrows():
    if row['tw_ids'] is not None:
        tw = smm.api.tweets_by(twitter_user_id=row['tw_ids'][0], aggregate_by='month')
        for i, date in enumerate(tw['labels'], 0):
            if date in search(row['affiliation'], party_list)['labels']:
                curr_i = search(row['affiliation'], party_list)['labels'].index(date)
                search(row['affiliation'], party_list)['values'][curr_i] += tw['values'][i]
            else:
                search(row['affiliation'], party_list)['labels'].append(date)
                search(row['affiliation'], party_list)['values'].append(tw['values'][i])

#print(party_list)

In [ ]:
from datetime import datetime
for element in party_list:
    dates = element['labels']
    values = element['values']
    dates = [datetime.strptime(item, '%Y-%m-%d') for item in dates]
    print(sorted(zip(dates,values)))
    


In [ ]:
#fix an error

import plotly
from plotly import graph_objs
dates = party_list[0]['labels']
values = party_list[0]['values']
data = sorted(zip(dates,values))
trace1 = [
    graph_objs.Scatter(
            data,
            mode='lines+markers', name=party_list[0]['party'],
            marker=dict(color='blue')
        )
    
]

layout = graph_objs.Layout(hovermode='closest',
                                   xaxis=dict(title='Date', ticklen=5,
                                              zeroline=True, gridwidth=2),
                                   yaxis=dict(title='Tweets',
                                              ticklen=5, gridwidth=2),
                                   legend=dict(x=0.5, y=1.2),
                                   showlegend=True)

data = trace1

plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot({"data": data, "layout": layout})